In [1]:
import networkx as nx
import seaborn as sns
import pandas as pd
import glob
import ast

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
filepath = r"C:\Users\Proprietario\Desktop\SLAPP\SLAPP3-master\6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX\Analisi 2" # use your path
all_files = glob.glob(filepath + "/*.txt")
G = {}
j = 0

for filename in all_files:
    G[j]=nx.DiGraph()
    fh=open(filename,'r')
    for i, line in enumerate(fh):
        s=line.strip()
        s=s.strip(";").split()
        if (i>6) and (s[0] != 'label') and (s[0] != '}') and (s[1] != '[style=filled,'):
            origin=int(s[0])
            dest=int(s[2])
            G[j].add_edge(origin,dest)
    fh.close()
    j += 1

In [3]:
for i in range (len(G)):
    print('The network has', len(G[i]), 'nodes and', len(G[i].edges()), 'links.')

The network has 98526 nodes and 2478149 links.
The network has 99881 nodes and 2885762 links.
The network has 99876 nodes and 3133781 links.
The network has 99952 nodes and 3625566 links.
The network has 99952 nodes and 3040681 links.
The network has 99869 nodes and 3291763 links.
The network has 99799 nodes and 2627940 links.
The network has 99975 nodes and 3756199 links.
The network has 99890 nodes and 2678180 links.


In [4]:
for i in range (len(G)):
    print("Is the graph simply connected?", nx.is_strongly_connected(G[i]))

Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False


### Show the components of the graph

In [5]:
for i in range (len(G)):
    print("The graph has", nx.number_strongly_connected_components(G[i]),"connected components")

The graph has 68515 connected components
The graph has 68197 connected components
The graph has 67640 connected components
The graph has 61781 connected components
The graph has 62043 connected components
The graph has 64068 connected components
The graph has 65492 connected components
The graph has 52155 connected components
The graph has 68301 connected components


### Extract the largest Connected Component as a subgraph

In [6]:
graphs = {}

for i in range (len(G)):
    graphs[i] = []
    for c in nx.strongly_connected_components(G[i]):
        graphs[i].append(G[i].subgraph(c))

In [7]:
H = {}

for i in range (len(G)):
    H[i] = graphs[i][0]

In [8]:
for i in range (len(G)):
    print(len(H[i]))

1
1
1
1
1
1
1
1
1


In [9]:
for i in range (len(G)):
    print(len(G[i])-len(H[i]))

98525
99880
99875
99951
99951
99868
99798
99974
99889


In [10]:
for i in range (len(G)):
    print("Check that the graph is now connected")
    print(nx.number_strongly_connected_components(H[i]))

Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1


## Clustering

In [11]:
files = glob.glob(filepath + "/*.csv")
dataset = {}
j = 0

for filename in files:
    dataset[j] = pd.read_csv(filename, encoding="utf8")
    j += 1

In [12]:
for i in range(len(dataset)):
    print(dataset[i].head)

<bound method NDFrame.head of         id_autore agent_type     score
0           17150     voters  0.442494
1           45191     voters  0.577299
2           80704  left_bots  0.886262
3            6955     voters  0.485468
4           85484     voters  0.631570
...           ...        ...       ...
99996       21817     voters  0.650869
99997       93518     voters  0.417721
99998       82690     voters  0.653285
99999       83032     voters  0.486040
100000      48208     voters  0.273558

[100001 rows x 3 columns]>
<bound method NDFrame.head of         id_autore agent_type     score
0           50636     voters  0.708286
1           85727     voters  0.217850
2           14069     voters  0.478620
3           52662     voters  0.409348
4           18265     voters  0.469141
...           ...        ...       ...
99996       28307     voters  0.409408
99997       13931     voters  0.485868
99998        8565     voters  0.489746
99999       99832     voters  0.147979
100000      998

In [13]:
N_nodes = {}
N_edges = {}
fake_news = {}
bias_right = {}
right = {}
right_leaning = {}
center_news = {}
left_leaning = {}
left = {}
bias_left = {}

for j in range (len(G)):
    N_nodes[str(j)] = [0]*8
    N_edges[str(j)] = [0]*8
    fake_news[j] = nx.DiGraph()
    bias_right[j] = nx.DiGraph()
    right[j] = nx.DiGraph()
    right_leaning[j] = nx.DiGraph()
    center_news[j] = nx.DiGraph()
    left_leaning[j] = nx.DiGraph()
    left[j] = nx.DiGraph()
    bias_left[j] = nx.DiGraph()
    
    for i in range (len(dataset[j])):
        if G[j].has_node(dataset[j]['id_autore'][i]):
            if dataset[j]['score'][i] < 0.1:
                N_nodes[str(j)][0] += 1
                N_edges[str(j)][0] += G[j].degree(dataset[j]['id_autore'][i])
                fake_news[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.2) and (dataset[j]['score'][i] > 0.1):
                N_nodes[str(j)][1] += 1
                N_edges[str(j)][1] += G[j].degree(dataset[j]['id_autore'][i])
                bias_right[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.3) and (dataset[j]['score'][i] > 0.2):
                N_nodes[str(j)][2] += 1
                N_edges[str(j)][2] += G[j].degree(dataset[j]['id_autore'][i])
                right[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.45) and (dataset[j]['score'][i] > 0.3):
                N_nodes[str(j)][3] += 1
                N_edges[str(j)][3] += G[j].degree(dataset[j]['id_autore'][i])
                right_leaning[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.55) and (dataset[j]['score'][i] > 0.45):
                N_nodes[str(j)][4] += 1
                N_edges[str(j)][4] += G[j].degree(dataset[j]['id_autore'][i])
                center_news[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.7) and (dataset[j]['score'][i] > 0.55):
                N_nodes[str(j)][5] += 1
                N_edges[str(j)][5] += G[j].degree(dataset[j]['id_autore'][i])
                left_leaning[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.8) and (dataset[j]['score'][i] > 0.7):
                N_nodes[str(j)][6] += 1
                N_edges[str(j)][6] += G[j].degree(dataset[j]['id_autore'][i])
                left[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.9) and (dataset[j]['score'][i] > 0.8):
                N_nodes[str(j)][7] += 1
                N_edges[str(j)][7] += G[j].degree(dataset[j]['id_autore'][i])
                bias_left[j].add_node(dataset[j]['id_autore'][i])

In [14]:
ok = True

In [15]:
if ok == True:
    for j in range (len(G)):
        for i in fake_news[j].node():
            for k in fake_news[j].node():
                if G[j].has_edge(i,k)==True:
                    fake_news[j].add_edge(i,k)

In [16]:
if ok == True:
    for j in range (len(G)):
        for i in bias_right[j].node():
            for k in bias_right[j].node():
                if G[j].has_edge(i,k)==True:
                    bias_right[j].add_edge(i,k)

In [17]:
if ok == True:
    for j in range (len(G)):
        for i in right[j].node():
            for k in right[j].node():
                if G[j].has_edge(i,k)==True:
                    right[j].add_edge(i,k)

In [18]:
if ok == True:
    for j in range (len(G)):
        for i in right_leaning[j].node():
            for k in right_leaning[j].node():
                if G[j].has_edge(i,k)==True:
                    right_leaning[j].add_edge(i,k)

In [19]:
if ok == True:
    for j in range (len(G)):
        for i in center_news[j].node():
            for k in center_news[j].node():
                if G[j].has_edge(i,k)==True:
                    center_news[j].add_edge(i,k)

In [20]:
if ok == True:
    for j in range (len(G)):
        for i in left_leaning[j].node():
            for k in left_leaning[j].node():
                if G[j].has_edge(i,k)==True:
                    left_leaning[j].add_edge(i,k)

In [21]:
if ok == True:
    for j in range (len(G)):
        for i in left[j].node():
            for k in left[j].node():
                if G[j].has_edge(i,k)==True:
                    left[j].add_edge(i,k)

In [22]:
if ok == True:
    for j in range (len(G)):
        for i in bias_left[j].node():
            for k in bias_left[j].node():
                if G[j].has_edge(i,k)==True:
                    bias_left[j].add_edge(i,k)

In [23]:
if ok == True:
    for j in range (len(G)):
        print('Fake-News Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(fake_news[j]),
              "connected components")

    for j in range (len(G)):
        print('Bias-Right Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(bias_right[j]),
              "connected components")

    for j in range (len(G)):
        print('Right Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(right[j]),
              "connected components")
    
    for j in range (len(G)):
        print('Right-Leaning Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(right_leaning[j]),
              "connected components")
    
    for j in range (len(G)):
        print('Center-News Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(center_news[j]),
              "connected components")
        
    for j in range (len(G)):
        print('Left-Leaning Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(left_leaning[j]),
              "connected components")
        
    for j in range (len(G)):
        print('Left Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(left[j]),
              "connected components")
        
    for j in range (len(G)):
        print('Bias-Left Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(bias_left[j]),
              "connected components")

Fake-News Networks:
The graph has 6992 connected components
Fake-News Networks:
The graph has 6788 connected components
Fake-News Networks:
The graph has 6530 connected components
Fake-News Networks:
The graph has 7153 connected components
Fake-News Networks:
The graph has 5548 connected components
Fake-News Networks:
The graph has 8245 connected components
Fake-News Networks:
The graph has 6071 connected components
Fake-News Networks:
The graph has 8235 connected components
Fake-News Networks:
The graph has 6097 connected components
Bias-Right Networks:
The graph has 8178 connected components
Bias-Right Networks:
The graph has 7959 connected components
Bias-Right Networks:
The graph has 7696 connected components
Bias-Right Networks:
The graph has 7972 connected components
Bias-Right Networks:
The graph has 7465 connected components
Bias-Right Networks:
The graph has 8710 connected components
Bias-Right Networks:
The graph has 7879 connected components
Bias-Right Networks:
The graph ha

In [24]:
K_mean = {}
voters_type = 8

for j in range (len(G)):
    K_mean[str(j)] = []
    print('\n')
    for i in range (voters_type):
        if N_nodes[str(j)][i] != 0:
            K_mean[str(j)].append(N_edges[str(j)][i]/N_nodes[str(j)][i])
            print("The ", i+1, " K_mean value is: ",K_mean[str(j)][i])
        else:
            K_mean[str(j)].append(str('Node do not exist'))
            print("The ", i+1, " K_mean value is: ",K_mean[str(j)][i])



The  1  K_mean value is:  112.37115603644646
The  2  K_mean value is:  69.84503279086714
The  3  K_mean value is:  81.95956734483647
The  4  K_mean value is:  51.796460176991154
The  5  K_mean value is:  33.461830316328395
The  6  K_mean value is:  42.74301329932592
The  7  K_mean value is:  36.452567693744165
The  8  K_mean value is:  27.419354838709676


The  1  K_mean value is:  123.11487949199018
The  2  K_mean value is:  113.55082332549213
The  3  K_mean value is:  93.50556438791733
The  4  K_mean value is:  54.68959225208786
The  5  K_mean value is:  38.2205805501449
The  6  K_mean value is:  41.95628882854061
The  7  K_mean value is:  55.49098338811406
The  8  K_mean value is:  25.652654867256636


The  1  K_mean value is:  174.29700725513905
The  2  K_mean value is:  95.02761089096255
The  3  K_mean value is:  68.16698499317872
The  4  K_mean value is:  51.7369098062954
The  5  K_mean value is:  54.85037477018809
The  6  K_mean value is:  40.970737445776045
The  7  K_mean val

In [25]:
import pydot
from networkx.drawing.nx_pydot import write_dot
oks = False

if oks == True:
    for j in range (1):
        write_dot(fake_news[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/fake-news' + str(j) + '.dot')

    for j in range (1):
        write_dot(bias_right[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/bias-right' + str(j) + '.dot')

    for j in range (1):
        write_dot(right[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/right' + str(j) + '.dot')

    for j in range (1):
        write_dot(right_leaning[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/right-leaning' + str(j) + '.dot')

    for j in range (1):
        write_dot(center_news[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/center-news' + str(j) + '.dot')

    for j in range (1):
        write_dot(left_leaning[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/left-leaning' + str(j) + '.dot')

    for j in range (1):
        write_dot(left[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/left' + str(j) + '.dot')

    for j in range (1):
        write_dot(bias_left[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/bias-left' + str(j) + '.dot')

In [26]:
nx.write_graphml_lxml(fake_news[0], "fake_news.graphml")
nx.write_graphml_lxml(bias_right[0], "bias_right.graphml")
nx.write_graphml_lxml(right[0], "right.graphml")
nx.write_graphml_lxml(right_leaning[0], "right_leaning.graphml")
nx.write_graphml_lxml(center_news[0], "center_news.graphml")
nx.write_graphml_lxml(left_leaning[0], "left_leaning.graphml")
nx.write_graphml_lxml(left[0], "left.graphml")
nx.write_graphml_lxml(bias_left[0], "bias_left.graphml")

In [27]:
for i in range (8):
    k_final = 0
    K_final = []
    
    for j in range (len(G)):
        K_final.append(K_mean[str(j)][i])
    print("The ", i+1, "k-final is: ", np.mean(K_final), "+-", np.std(K_final))

The  1 k-final is:  147.41928319004938 +- 35.41897054417802
The  2 k-final is:  92.1746585258276 +- 18.282196282684374
The  3 k-final is:  77.93674208470352 +- 13.475653590971708
The  4 k-final is:  54.68692325455497 +- 10.71862063745451
The  5 k-final is:  46.6411773995812 +- 7.652151650748298
The  6 k-final is:  41.78168961478068 +- 5.629098663451183
The  7 k-final is:  57.42197241057233 +- 9.933222090492738
The  8 k-final is:  84.9723576989415 +- 46.99696084197339


In [28]:
for i in range (len(G)):
    print("Simulation number: ", i+1)
    print(fake_news[i].number_of_edges()/fake_news[i].number_of_nodes())
    print(bias_right[i].number_of_edges()/bias_right[i].number_of_nodes())
    print(right[i].number_of_edges()/right[i].number_of_nodes())
    print(right_leaning[i].number_of_edges()/right_leaning[i].number_of_nodes())
    print(center_news[i].number_of_edges()/center_news[i].number_of_nodes())
    print(left_leaning[i].number_of_edges()/left_leaning[i].number_of_nodes())
    print(left[i].number_of_edges()/left[i].number_of_nodes())
    print(bias_left[i].number_of_edges()/bias_left[i].number_of_nodes())
    print()

Simulation number:  1
6.305523917995444
3.8843818314306535
4.512232809683234
3.697798581145323
1.8221795575059425
4.949280378939697
1.3
0.06

Simulation number:  2
6.981959878770385
7.200817135074904
4.584043936985114
3.660958663765878
1.9068364292840514
3.606115843904617
3.2021853069201387
0.022345132743362833

Simulation number:  3
9.869407496977026
5.385657676083344
2.7759890859481584
2.6643462469733654
4.980577947485033
2.680869053746048
3.1827005669129016
2.015932133250569

Simulation number:  4
12.364385820280297
6.24620323496728
2.661988476879894
4.055353961865805
3.962526614620298
2.9905062103643663
3.1661080074487895
2.9532121212121214

Simulation number:  5
4.289813486370158
4.760884036746106
4.661756500390581
1.5108498129342598
6.058191545342923
2.2691523755974137
4.32526026604974
4.018764153995471

Simulation number:  6
10.763331336129419
7.972323879231473
2.5848774656306035
3.885484622553588
3.1848334580838324
3.02569243335561
2.1798080427651563
0.03774442928603911

Simula

In [42]:
fake_mean = []
bias_right_mean = []
right_mean = []
right_leaning_mean = []
center_news_mean = []
left_leaning_mean = []
left_mean = []
bias_left_mean = []

for i in range (len(G)):
    if (i != 0) and (i != 3) and (i != 5) and (i != 7):
        fake_mean.append(fake_news[i].number_of_edges()/fake_news[i].number_of_nodes())
        bias_right_mean.append(bias_right[i].number_of_edges()/bias_right[i].number_of_nodes())
        right_mean.append(right[i].number_of_edges()/right[i].number_of_nodes())
        right_leaning_mean.append(right_leaning[i].number_of_edges()/right_leaning[i].number_of_nodes())
        center_news_mean.append(center_news[i].number_of_edges()/center_news[i].number_of_nodes())
        left_leaning_mean.append(left_leaning[i].number_of_edges()/left_leaning[i].number_of_nodes())
        left_mean.append(left[i].number_of_edges()/left[i].number_of_nodes())
        bias_left_mean.append(bias_left[i].number_of_edges()/bias_left[i].number_of_nodes())
    
print(mean(fake_mean), '+-' , std(fake_mean))
print(mean(bias_right_mean), '+-' , std(bias_right_mean))
print(mean(right_mean), '+-' , std(right_mean))
print(mean(right_leaning_mean), '+-' , std(right_leaning_mean))
print(mean(center_news_mean), '+-' , std(center_news_mean))
print(mean(left_leaning_mean), '+-' , std(left_leaning_mean))
print(mean(left_mean), '+-' , std(left_mean))
print(mean(bias_left_mean), '+-' , std(bias_left_mean))

6.723697187177696 +- 1.920993904340087
4.982171475291265 +- 1.2662025431056343
3.659793635623032 +- 0.8705697551685001
2.5927999937155923 +- 1.441142537822679
4.052735292118099 +- 1.422030850040719
2.562197983109132 +- 0.5602806444507218
3.5161637923000155 +- 0.42312443169833924
2.2174210573817135 +- 1.3236957833602239
